![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace%20in%20Spark%20NLP%20-%20DeBertaForTokenClassification.ipynb)

## Import DeBertaForTokenClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only in `Spark NLP 3.4.4` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import DeBerta models trained/fine-tuned for token classification via `DeBertaForTokenClassification` or `TFDebertaV2ForTokenClassification`. These models are usually under `Token Classification` category and have `deberta` in their labels
- Reference: [TFDebertaV2ForTokenClassification](https://huggingface.co/docs/transformers/model_doc/deberta-v2#transformers.TFDebertaV2ForSequenceClassification)
- Some [example models](https://huggingface.co/models?other=deberta-v2&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.25.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- DebertaV2Tokenizer requires the `SentencePiece` library, so we install that as well

In [1]:
!pip install -q transformers==4.25.1 tensorflow==2.11.0 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 890.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 40.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requ

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [Gladiator/microsoft-deberta-v3-large_ner_conll2003](https://huggingface.co/Gladiator/microsoft-deberta-v3-large_ner_conll2003) model from HuggingFace as an example
- In addition to `TFDebertaV2ForTokenClassification` we also need to save the `DebertaV2Tokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [3]:
from transformers import TFDebertaV2ForTokenClassification, DebertaV2Tokenizer
import tensorflow as tf

MODEL_NAME = 'Gladiator/microsoft-deberta-v3-large_ner_conll2003'

tokenizer = DebertaV2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFDebertaV2ForTokenClassification.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFDebertaV2ForTokenClassification.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask"),
          "token_type_ids": tf.TensorSpec((None, None), tf.int32, name="token_type_ids")

      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})


spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/400 [00:00<?, ?B/s]

try downloading TF weights


config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDebertaV2ForTokenClassification: ['deberta.embeddings.position_ids']
- This IS expected if you are initializing TFDebertaV2ForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDebertaV2ForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDebertaV2ForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDebertaV2ForTokenClassification for predictions without further training.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlo

Let's have a look inside these two directories and see what we are dealing with:

In [4]:
!ls -l {MODEL_NAME}

total 1695948
-rw-r--r-- 1 root root       1246 Jan 15 18:40 config.json
drwxr-xr-x 3 root root       4096 Jan 15 18:40 saved_model
-rw-r--r-- 1 root root 1736635840 Jan 15 18:40 tf_model.h5


In [5]:
!ls -l {MODEL_NAME}/saved_model/1

total 49168
drwxr-xr-x 2 root root     4096 Jan 15 18:40 assets
-rw-r--r-- 1 root root       54 Jan 15 18:40 fingerprint.pb
-rw-r--r-- 1 root root   346522 Jan 15 18:40 keras_metadata.pb
-rw-r--r-- 1 root root 49987357 Jan 15 18:40 saved_model.pb
drwxr-xr-x 2 root root     4096 Jan 15 18:40 variables


In [6]:
!ls -l {MODEL_NAME}_tokenizer

total 2420
-rw-r--r-- 1 root root      23 Jan 15 18:34 added_tokens.json
-rw-r--r-- 1 root root     173 Jan 15 18:34 special_tokens_map.json
-rw-r--r-- 1 root root 2464616 Jan 15 18:34 spm.model
-rw-r--r-- 1 root root     479 Jan 15 18:34 tokenizer_config.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `spm.model` file from the tokenizer
- all we need is to copy `spm.model` file into `saved_model/1/assets` which Spark NLP will look for
- in addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [7]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

# let's copy spm.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/spm.model {asset_path}

In [8]:
# get label2id dictionary
labels = model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [9]:
! ls -l {asset_path}

total 2412
-rw-r--r-- 1 root root      51 Jan 15 18:41 labels.txt
-rw-r--r-- 1 root root 2464616 Jan 15 18:41 spm.model


In [10]:
#Restart Session here to clear up RAM
1+while:

SyntaxError: invalid syntax (<ipython-input-10-eb7b17df1069>, line 2)

## Import and Save DeBertaForTokenClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [3]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.2.2
setup Colab for PySpark 3.2.3 and Spark NLP 5.2.2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.3/547.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 12.0 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [4]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `DeBertaForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DeBertaForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [5]:
from sparknlp.annotator import *
from sparknlp.base import *

MODEL_NAME = 'Gladiator/microsoft-deberta-v3-large_ner_conll2003'

tokenClassifier = DeBertaForTokenClassification\
  .loadSavedModel('{}/saved_model/1'.format(MODEL_NAME), spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [6]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [7]:
! rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your DeBertaForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [8]:
! ls -l {MODEL_NAME}_spark_nlp

total 1746372
-rw-r--r-- 1 root root 1785805765 Jan 15 18:52 deberta_classification_tensorflow
-rw-r--r-- 1 root root    2464616 Jan 15 18:52 deberta_spp
drwxr-xr-x 4 root root       4096 Jan 15 18:46 fields
drwxr-xr-x 2 root root       4096 Jan 15 18:46 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny DeBertaForTokenClassification model 😊

In [1]:
1+while
#restart here

SyntaxError: invalid syntax (<ipython-input-1-6f8c84fe2791>, line 1)

In [3]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

In [4]:
from sparknlp.annotator import *
from sparknlp.base import *

MODEL_NAME = 'Gladiator/microsoft-deberta-v3-large_ner_conll2003'

tokenClassifier_loaded = DeBertaForTokenClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [5]:
# .getClasses was introduced in spark-nlp==3.4.0
tokenClassifier_loaded.getClasses()

['B-LOC', 'I-ORG', 'I-MISC', 'I-LOC', 'I-PER', 'B-MISC', 'B-ORG', 'O', 'B-PER']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [10]:
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    tokenClassifier_loaded
])

# couple of simple examples
example = spark.createDataFrame([["My name is Wolfgang and I live in Berlin"]]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "ner.result").show(truncate=False)

+----------------------------------------+-----------------------------------+
|text                                    |result                             |
+----------------------------------------+-----------------------------------+
|My name is Wolfgang and I live in Berlin|[O, O, O, B-PER, O, O, O, O, B-LOC]|
+----------------------------------------+-----------------------------------+



That's it! You can now go wild and use hundreds of `DeBertaForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀
